# Generating Music

In [7]:
import numpy as np
import random
import glob
from tqdm import tqdm
from music21 import *
from tensorflow.keras.layers import LSTM,Dense,Input,Dropout
from tensorflow.keras.models import Sequential,Model,load_model
from sklearn.model_selection import train_test_split

In [65]:
def read_files(file):
  notes=[]
  notes_to_parse=None
  #parse the midi file
  midi=converter.parse(file)
  #seperate all instruments from the file
  instrmt=instrument.partitionByInstrument(midi)

  for part in instrmt.parts:
    #fetch data only of Piano instrument
    if 'Piano' in str(part):
      notes_to_parse=part.recurse()

      #iterate over all the parts of sub stream elements
      #check if element's type is Note or chord
      #if it is chord split them into notes
      for element in notes_to_parse:
        if type(element)==note.Note:
          notes.append(str(element.pitch))
        elif type(element)==chord.Chord:
          notes.append('.'.join(str(n) for n in element.normalOrder))

  #return the list of notes
  return np.array(notes)

In [67]:
#specify the path
path='Data/schubert/'

#read all the filenames
files=[i for i in os.listdir(path) if i.endswith(".mid")]

#reading each midi file
notes_array = np.array([read_midi(path+i) for i in files])

Loading Music File: Data/schubert/schubert_D850_1.mid
Loading Music File: Data/schubert/schubert_D850_2.mid
Loading Music File: Data/schubert/schubert_D850_3.mid
Loading Music File: Data/schubert/schubert_D850_4.mid
Loading Music File: Data/schubert/schubert_D935_1.mid
Loading Music File: Data/schubert/schubert_D935_2.mid
Loading Music File: Data/schubert/schubert_D935_3.mid
Loading Music File: Data/schubert/schubert_D935_4.mid
Loading Music File: Data/schubert/schub_d760_1.mid
Loading Music File: Data/schubert/schub_d760_2.mid
Loading Music File: Data/schubert/schub_d760_3.mid
Loading Music File: Data/schubert/schub_d760_4.mid
Loading Music File: Data/schubert/schub_d960_1.mid
Loading Music File: Data/schubert/schub_d960_2.mid
Loading Music File: Data/schubert/schub_d960_3.mid
Loading Music File: Data/schubert/schub_d960_4.mid
Loading Music File: Data/schubert/schuim-1.mid
Loading Music File: Data/schubert/schuim-2.mid
Loading Music File: Data/schubert/schuim-3.mid
Loading Music File:

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (29,) + inhomogeneous part.

In [47]:
#retrieve paths recursively from inside the directories/files
file_path=["muss"]
all_files=glob.glob('Data/'+file_path[0]+'/*.mid',recursive=True)

In [49]:
notes_array = [read_files(file) for file in tqdm(all_files,position=0,leave=True)]

100%|████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:05<00:00,  1.40it/s]


In [51]:
np.array(notes_array)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (8,) + inhomogeneous part.

In [50]:
max_length = max([len(notes_array[i]) for i in range(0, len(notes_array))])
min_length = min([len(notes_array[i]) for i in range(0, len(notes_array))])
[len(notes_array[i]) for i in range(0, len(notes_array))]

[950, 1054, 518, 447, 775, 583, 2187, 2750]

In [33]:
# Pad sublists with '0' to make them uniform in length
padded_list = [sublist + ['0'] * (max_length - len(sublist)) for sublist in notes_array]
padded_array = np.array(padded_list)
#notess = sum(padded_array,[])

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U14')) -> None

In [ ]:
#unique notes
notess = sum(notes_array,[]) 
unique_notes = list(set(notess))
print("Unique Notes:",len(unique_notes))

In [ ]:
#notes with their frequency
freq=dict(map(lambda x: (x,notess.count(x)),unique_notes))

#get the threshold frequency
print("\nFrequency notes")
for i in range(30,100,20):
  print(i,":",len(list(filter(lambda x:x[1]>=i,freq.items()))))

#filter notes greater than threshold i.e. 50
freq_notes=dict(filter(lambda x:x[1]>=50,freq.items()))

#create new notes using the frequent notes
new_notes=[[i for i in j if i in freq_notes] for j in notes_array]

#dictionary having key as note index and value as note
ind2note=dict(enumerate(freq_notes))

#dictionary having key as note and value as note index
note2ind=dict(map(reversed,ind2note.items()))

#timestep
timesteps=50

#store values of input and output
x=[] ; y=[]

In [53]:
def read_files(file):
    notes = []
    notes_to_parse = None
    
    # Parse the midi file
    midi = converter.parse(file)
    
    # Separate all instruments from the file
    instrmt = instrument.partitionByInstrument(midi)

    for part in instrmt.parts:
        # Fetch data only for Piano instrument
        if 'Piano' in str(part):
            notes_to_parse = part.recurse()

            # Iterate over all the parts of sub stream elements
            # Check if element's type is Note or chord
            # If it is chord, split them into notes
            for element in notes_to_parse:
                try:
                    if type(element) == note.Note:
                        notes.append(str(element.pitch))
                    elif type(element) == chord.Chord:
                        notes.append('.'.join(str(n) for n in element.normalOrder))
                except AttributeError as e:
                    # Handle cases where instrument information is not available
                    print(f"Warning: {e}")
                    pass

    # Return the list of notes
    return notes

In [54]:
#retrieve paths recursively from inside the directories/files
file_path=["mozart"]
all_files=glob.glob('Data/'+file_path[0]+'/*.mid',recursive=True)

#reading each midi file
notes_array = [read_files(i) for i in tqdm(all_files,position=0,leave=True)]

 28%|██████████████████████▉                                                            | 8/29 [09:45<25:37, 73.22s/it]
C:\Users\RaviB\anaconda3\envs\HarmonAIze\Lib\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
 14%|███████████▊                                                                       | 3/21 [00:15<01:36,  5.36s/it]C:\Users\RaviB\anaconda3\envs\HarmonAIze\Lib\site-packages\music21\midi\translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
 29%|███████████████████████▋                                                           | 6/21 [00:30<01:16,  5.07s/it]C:\Users\RaviB\anaconda3\envs\Ha